In [5]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from statsmodels.stats.contingency_tables import mcnemar
from dataset_tools import OwnDataset, make_train_test_val_split, make_small_subset
from sklearn.metrics import confusion_matrix
from models import LatentClassifier_2, VAE_CELL_CNN, CELL_CNN_CLASSIFIER
from torchvision import transforms

cuda = torch.cuda.is_available()
print(cuda)

True


In [6]:
tf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

# dataset = OwnDataset(transform=tf, path=r"/Users/rasmusjensen/Documents/02466-Cell-Profiling/data_subset")
dataset = OwnDataset(transform=tf, path=r"C:\Users\Otto\Desktop\Fagprojekt_data\labelled_data")
train_subset, test_subset, val_subset = make_train_test_val_split(dataset)
# train_subset = make_small_subset(dataset, 100000)

train_loader = DataLoader(train_subset, batch_size=len(train_subset), shuffle=True)
test_loader = DataLoader(test_subset, batch_size=len(test_subset), shuffle=True)
val_loader = DataLoader(val_subset, batch_size=len(val_subset), shuffle=True)

print(len(dataset), len(train_loader), len(test_loader), len(val_loader))


488396 1 1 1


In [7]:
MODEL_DIR = "trained_models/"


latent_dim_2 = LatentClassifier_2(hidden_dim=4, latent_dim=2, num_classes=13)
latent_dim_2.load_state_dict(torch.load(MODEL_DIR + f"{latent_dim_2.__class__.__name__}_latent{latent_dim_2.latent_dim}_mean_best_model.pth", map_location=torch.device('cpu')))
latent_dim_2.eval()


latent_dim_32 = LatentClassifier_2(hidden_dim=64, latent_dim=32, num_classes=13)
latent_dim_32.load_state_dict(torch.load(MODEL_DIR + f"{latent_dim_32.__class__.__name__}_latent{latent_dim_32.latent_dim}_mean_best_model.pth", map_location=torch.device('cpu')))
latent_dim_32.eval()


latent_dim_256 = LatentClassifier_2(hidden_dim=512, latent_dim=256, num_classes=13)
latent_dim_256.load_state_dict(torch.load(MODEL_DIR + f"{latent_dim_256.__class__.__name__}_latent{latent_dim_256.latent_dim}_mean_best_model.pth", map_location=torch.device('cpu')))
latent_dim_256.eval()


Vae_Cell_CNN_2 = VAE_CELL_CNN(input_dim=(3, 68, 68), hidden_dim=4, latent_dim=2)
Vae_Cell_CNN_2.load_state_dict(torch.load(MODEL_DIR + f"{Vae_Cell_CNN_2.__class__.__name__}_latent{Vae_Cell_CNN_2.latent_dim}_mean_best_model.pth", map_location=torch.device('cpu')))
Vae_Cell_CNN_2.eval()


Vae_Cell_CNN_32 = VAE_CELL_CNN(input_dim=(3, 68, 68), hidden_dim=64, latent_dim=32)
Vae_Cell_CNN_32.load_state_dict(torch.load(MODEL_DIR + f"{Vae_Cell_CNN_32.__class__.__name__}_latent{Vae_Cell_CNN_32.latent_dim}_mean_best_model.pth", map_location=torch.device('cpu')))
Vae_Cell_CNN_32.eval()


Vae_Cell_CNN_256 = VAE_CELL_CNN(input_dim=(3, 68, 68), hidden_dim=512, latent_dim=256)
Vae_Cell_CNN_256.load_state_dict(torch.load(MODEL_DIR + f"{Vae_Cell_CNN_256.__class__.__name__}_latent{Vae_Cell_CNN_256.latent_dim}_mean_best_model.pth", map_location=torch.device('cpu')))
Vae_Cell_CNN_256.eval()

Cell_CNN_Classifier = CELL_CNN_CLASSIFIER(input_dim=(3, 68, 68), hidden_dim=128, num_classes=13)
Cell_CNN_Classifier.load_state_dict(torch.load(MODEL_DIR + f"{Cell_CNN_Classifier.__class__.__name__}_best_model.pth", map_location=torch.device('cpu')))
Cell_CNN_Classifier.eval()
# 
models = [latent_dim_256, latent_dim_2, Cell_CNN_Classifier]


In [8]:
images, moas = next(iter(test_loader)) # this is the whole dataset, as batch size is the same as the dataset size
print(len(images), len(moas))

with torch.no_grad():
    latent_output_256 = Vae_Cell_CNN_256(images)
    pred_256 = latent_dim_256(latent_output_256["mu"])
    accuracy_latent_256 = (pred_256.argmax(1) == moas).cpu().numpy()

    latent_output_2 = Vae_Cell_CNN_2(images)
    pred_2 = latent_dim_2(latent_output_2["mu"])
    accuracy_latent_2 = (pred_2.argmax(1) == moas).cpu().numpy()

    latent_output_32 = Vae_Cell_CNN_32(images)
    pred_32 = latent_dim_32(latent_output_32["mu"])
    accuracy_latent_32 = (pred_32.argmax(1) == moas).cpu().numpy()

    pred_CNN = Cell_CNN_Classifier(images)
    accuracy_CNN = (pred_CNN.argmax(1) == moas).cpu().numpy()


    

97680 97680


In [9]:
np.mean(accuracy_latent_256), np.mean(accuracy_latent_2), np.mean(accuracy_latent_32), np.mean(accuracy_CNN)

(0.7450143325143325,
 0.6475634725634726,
 0.7443079443079443,
 0.8504811629811629)

In [10]:
def compute_mcnemar(accuracy_1, accuracy_2):
    # 2x2 confusion matrix for the two models
    confus = confusion_matrix(accuracy_1, accuracy_2)

    result = mcnemar(confus)

    print(confus)

    print(result)

    return result, confus

In [11]:
result, confus = compute_mcnemar(accuracy_latent_256, accuracy_latent_2)

result, confus = compute_mcnemar(accuracy_latent_256, accuracy_latent_32)

result, confus = compute_mcnemar(accuracy_latent_256, accuracy_CNN)



[[22215  2692]
 [12211 60562]]
pvalue      0.0
statistic   2692.0
[[22046  2861]
 [ 2930 69843]]
pvalue      0.3715494820511749
statistic   2861.0
[[12158 12749]
 [ 2447 70326]]
pvalue      0.0
statistic   2447.0


In [12]:
# def compute_mcnemar(acc1, acc2):
#     n00 = n01 = n10 = n11 = 0
#     for i in range(len(acc1)):
#         if acc1[i] and acc2[i]:
#             n11 += 1
#         elif acc1[i] and not acc2[i]:
#             n10 += 1
#         elif not acc1[i] and acc2[i]:
#             n01 += 1
#         else:
#             n00 += 1
#     confusion_matrix = [[n00, n01], [n10, n11]]
#     print(f'Confusion Matrix: {confusion_matrix}')
#     result = mcnemar(confusion_matrix, exact=True)
#     print(f'statistic={result.statistic}, p-value={result.pvalue}')
#     if result.pvalue < 0.05:
#         print('Significant difference in performance (reject H0)')
#     else:
#         print('No significant difference in performance (fail to reject H0)')


In [13]:
# # Perform McNemar's tests
# print("CNN vs Latent 256")
# print("----------------")
# compute_mcnemar(batch_accuracy_CNN, batch_accuracy_256_latent)
# print(" ")

# print("Latent 256 vs Latent 2")
# print("----------------")
# compute_mcnemar(batch_accuracy_256_latent, batch_accuracy_2_latent)
# print(" ")

# print("Latent 2 vs CNN")
# print("----------------")
# compute_mcnemar(batch_accuracy_2_latent, batch_accuracy_CNN)
# print(" ")


In [14]:
# from statsmodels.stats.contingency_tables import mcnemar

# def compute_mcnemar(acc1, acc2):
#     n00 = n01 = n10 = n11 = 0
#     for i in range(len(acc1)):
#         if acc1[i] == 1 and acc2[i] == 1:
#             n11 += 1
#         elif acc1[i] == 1 and acc2[i] == 0:
#             n10 += 1
#         elif acc1[i] == 0 and acc2[i] == 1:
#             n01 += 1
#         else:
#             n00 += 1
#     confusion_matrix = [[n00, n01], [n10, n11]]
#     print(f'Confusion Matrix: {confusion_matrix}')
#     result = mcnemar(confusion_matrix, exact=True)
#     print(f'statistic={result.statistic}, p-value={result.pvalue}')
#     if result.pvalue < 0.05:
#         print('Significant difference in performance (reject H0)')
#     else:
#         print('No significant difference in performance (fail to reject H0)')


# #acc1 = [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]  
# #acc2 = [1, 0, 1, 0, 0, 1, 0, 0, 1, 1]  
# #compute_mcnemar(acc1, acc2)

# compute_mcnemar(batch_accuracy_CNN, batch_accuracy_256_latent)


In [15]:
# compute_mcnemar(batch_accuracy_CNN, batch_accuracy_256_latent)